In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
lines = open("/kaggle/input/movies/movie_lines.txt", encoding="ISO-8859-1").read().split('\n')
lines

In [ ]:
conversation = open("/kaggle/input/movies/movie_conversations.txt", encoding="ISO-8859-1").read().split('\n')
conversation

# Preprocess

In [ ]:

exch=[]
for conv in conversation:
    exch.append(conv.split(" +++$+++ ")[-1][1:-1].replace("'","").replace(",","").split())

In [ ]:
exch

In [ ]:
dialogues={}
for line in lines:
    dialogues[line.split(' +++$+++ ')[0]]=[line.split('+++$+++')[-1]]

In [ ]:
dialogues

In [ ]:
questions=[]
answers=[]

In [ ]:
for convers in exch:
    for i in range(len(convers)-1):
        questions.append(dialogues[convers[i]])
        answers.append(dialogues[convers[i+1]])

In [ ]:
del(exch,dialogues,line,conversation,i,lines,conv,convers,)

In [ ]:
questions

In [ ]:
answers

In [ ]:
s_question=[]
s_answer=[]
for i in range(len(questions)):
    if len(questions[i][0]) < 13:
        s_question.append(questions[i])
        s_answer.append(answers[i])

In [ ]:
s_question
s_question = sum(s_question, [])

In [ ]:
len(s_answer)
len(s_question)
s_answer = sum(s_answer, [])

In [ ]:
import re
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [ ]:
clean_question=[clean_text(s_question[i]) for i in range(len(s_question))]
clean_answer=[clean_text(s_answer[i]) for i in range(len(s_answer))]

In [ ]:
print(len(clean_question))
print(len(clean_answer))

In [ ]:
for i in range(len(clean_answer)):
    clean_answer[i]=" ".join(clean_answer[i].split()[:11])

In [ ]:
del(i,s_answer,s_question,questions,answers)

In [ ]:
clean_answer

In [ ]:
print(len(clean_answer))
print(len(clean_question))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Count occurrences using Keras Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_question + clean_answer)

# Remove less frequent words and create vocab
thresh = 5
vocab_size = len([word for word, count in tokenizer.word_counts.items() if count >= thresh])

tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OUT>')
tokenizer.fit_on_texts(clean_question + clean_answer)

# Preprocess answers by adding start and end tokens
clean_ans = ['<SOS> ' + ans + ' <EOS>' for ans in clean_answer]

# Extend vocabulary for special tokens
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
    tokenizer.word_index[token] = vocab_size
    vocab_size += 1

# Swap values in the vocabulary
tokenizer.word_index['cameron'] = tokenizer.word_index['<PAD>']
tokenizer.word_index['<PAD>'] = 0

# Create inverse vocabulary mapping
inv_vocab = {v: w for w, v in tokenizer.word_index.items()}

# Convert words to indices
encoder_inp = tokenizer.texts_to_sequences(clean_question)
decoder_inp = tokenizer.texts_to_sequences(clean_answer)

# Pad sequences
MAX_LEN = 13
encoder_inp = pad_sequences(encoder_inp, maxlen=MAX_LEN, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, maxlen=MAX_LEN, padding='post', truncating='post')

# Create decoder final output
decoder_final_output = [seq[1:] for seq in decoder_inp]
decoder_final_output = pad_sequences(decoder_final_output, maxlen=MAX_LEN, padding='post', truncating='post')

# One-hot encode decoder final output
decoder_final_output = to_categorical(decoder_final_output, num_classes=vocab_size)

In [ ]:
from keras.layers import Embedding,Dense,LSTM,Input
from keras.models import Model


enc_input=Input(shape=(13,))
dec_input=Input(shape=(13,))

embed=Embedding(vocab_size+1,output_dim=50,input_length=13,trainable=True)

enc_embed=embed(enc_input)
enc_lstm=LSTM(64,return_sequences=True,return_state=True)
enc_op,h,c=enc_lstm(enc_embed)
enc_states=[h,c]

dec_embed=embed(dec_input)
dec_lstm=LSTM(64,return_sequences=True,return_state=True)
dec_op,_,_=dec_lstm(dec_embed,initial_state=enc_states)
dense=Dense(vocab_size,activation='softmax')
dense_op=dense(dec_op)

model=Model([enc_input,dec_input],dense_op)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',  
              metrics=['accuracy'])

In [ ]:
history=model.fit(
    x=[encoder_inp, decoder_inp],  # Pass the input data as a list if you have multiple inputs
    y=decoder_final_output,         # Replace with your actual target data
    epochs=50
)

In [ ]:
import matplotlib.pyplot as plt



plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.xlabel("Epochs")

plt.legend(['Loss','accuracy'],loc='upper right')
plt.show()

In [ ]:
model.save('chatbot_real.h5')
model.save_weights('chatbot_weights_real.h5')

In [49]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


enc_model = Model([enc_input], enc_states)



# decoder Model
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]


dec_model = Model([dec_input]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)




from keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    prepro1  = input("you : ")
    ## prepro1 = "Hello"

    prepro1 = clean_text(prepro1)
    ## prepro1 = "hello"

    prepro = [prepro1]
    ## prepro1 = ["hello"]

    txt = []
    for x in prepro:
        # x = "hello"
        lst = []
        for y in x.split():
            ## y = "hello"
            try:
                lst.append(vocab[y])
                ## vocab['hello'] = 454
            except:
                lst.append(vocab['<OUT>'])
        txt.append(lst)

    ## txt = [[454]]
    txt = pad_sequences(txt, 13, padding='post')

    ## txt = [[454,0,0,0,.........13]]

    stat = enc_model.predict( txt )

    empty_target_seq = np.zeros( ( 1 , 1) )
     ##   empty_target_seq = [0]


    empty_target_seq[0, 0] = vocab['<SOS>']
    ##    empty_target_seq = [255]

    stop_condition = False
    decoded_translation = ''

    while not stop_condition :

        dec_outputs , h, c= dec_model.predict([ empty_target_seq] + stat )
        decoder_concat_input = dense(dec_outputs)
        ## decoder_concat_input = [0.1, 0.2, .4, .0, ...............]

        sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
        ## sampled_word_index = [2]

        sampled_word = inv_vocab[sampled_word_index] + ' '

        ## inv_vocab[2] = 'hi'
        ## sampled_word = 'hi '

        if sampled_word != '<EOS> ':
            decoded_translation += sampled_word  

        if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
            stop_condition = True 

        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        ## <SOS> - > hi
        ## hi --> <EOS>
        stat = [h, c]  

    print("chatbot attention : ", decoded_translation )
    print("==============================================")  
















##########################################
#       start chatting ver. 1.0          #
##########################################


you :  hello


1/1 [==============================] - 0s 19ms/step
chatbot attention :  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


you :  hiw are you?


1/1 [==============================] - 0s 19ms/step
chatbot attention :  thousands thousands <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


...

1/1 [==============================] - 0s 21ms/step
chatbot attention :  thousands thousands <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


you :  drink anything to the world


1/1 [==============================] - 0s 20ms/step
chatbot attention :  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 


you :  q


1/1 [==============================] - 0s 19ms/step
chatbot attention :  <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 
